Adesso hai già:

un motore di ranking + spiegazioni (rank_and_explain(query, top_k)) definito nello Step D,

che prende una descrizione tipo:

"Cerco una guardia tiratrice con ottime percentuali da 3, buon FT%, forte rimbalzista offensivo e buon difensore."

e restituisce:

un DataFrame con i giocatori ordinati per similarità,

una colonna explanation con una spiegazione in linguaggio naturale.

👉 Con lo Step E vogliamo costruire una piccola interfaccia da Talent Scout, in cui:

l’utente scrive da tastiera la descrizione del giocatore ideale,

il sistema risponde con una shortlist top-N + spiegazioni stampate in modo leggibile.

In [16]:
import numpy as np
import pandas as pd
import re
from sklearn.metrics.pairwise import cosine_similarity
import os
import joblib
import pandas as pd

# === PATH (stessi che hai usato negli step B e D) ===
MODELS_DIR = "../data/models"
DATA_CLEAN_PATH = "../data/drafted_cleaned.csv"

VECTORIZER_PATH = os.path.join(MODELS_DIR, "tfidf_vectorizer.joblib")
MATRIX_PATH = os.path.join(MODELS_DIR, "tfidf_matrix.joblib")
META_PATH = os.path.join(MODELS_DIR, "index_metadata.csv")

# === Carica indice TF-IDF ===
vectorizer = joblib.load(VECTORIZER_PATH)
tfidf_matrix = joblib.load(MATRIX_PATH)
metadata_df = pd.read_csv(META_PATH)

print("Indice caricato.")
print("TF-IDF matrix shape:", tfidf_matrix.shape)
print("Metadata shape:", metadata_df.shape)

# === Carica dataset completo con stats (stesso della parte 1) ===
full_df = pd.read_csv(DATA_CLEAN_PATH)
print("Dataset stats shape:", full_df.shape)


# =========================================
# 1. Normalizzazione testo
# =========================================

def normalize_text(text: str) -> str:
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = re.sub(r"\s+", " ", text).strip()
    return text

# =========================================
# 2. Skill patterns (NO RUOLI)
# =========================================

SKILL_PATTERNS = {
    "shooting_3": ["tiratore da 3", "percentuali da 3", "da 3 punti", "tripla", "3 punti"],
    "ft": ["tiri liberi", "ft%", "liberi"],
    "reb": ["rimbalzista", "rimbalzi"],
    "reb_off": ["rimbalzo offensivo", "rimbalzista offensivo"],
    "playmaking": ["assist", "passatore", "playmaker", "creatore di gioco"],
    "defense": ["difensore", "difesa", "stoppate", "recuperi"],
    "scorer": ["realizzatore", "tanti punti", "scorer"]
}

SEMANTIC_EXPANSIONS = {
    "shooting_3": "ottimo tiratore da 3 punti tiro da 3 3P% buon tiratore da 3",
    "ft": "buon tiratore di tiri liberi FT% tiri liberi",
    "reb": "rimbalzi forte rimbalzista",
    "reb_off": "rimbalzo offensivo rimbalzista offensivo",
    "playmaking": "assist buon passatore playmaker",
    "defense": "difensore buon difensore stoppate recuperi",
    "scorer": "punti realizzatore"
}

SEMANTIC_WEIGHTS = {
    "shooting_3": 4,
    "ft": 3,
    "reb": 3,
    "reb_off": 4,
    "playmaking": 3,
    "defense": 3,
    "scorer": 3
}

# =========================================
# 3. Parsing intenti
# =========================================

def parse_query_intents(raw_query: str):
    q = normalize_text(raw_query)
    detected_skills = set()

    for skill, patterns in SKILL_PATTERNS.items():
        for p in patterns:
            if p in q:
                detected_skills.add(skill)
                break

    return {
        "normalized_query": q,
        "roles": [],   # ruoli disattivati
        "skills": sorted(list(detected_skills))
    }

# =========================================
# 4. Query pesata
# =========================================

def build_weighted_query(raw_query: str, intents: dict) -> str:
    base = intents.get("normalized_query", normalize_text(raw_query))
    boosted = [base]

    for skill in intents.get("skills", []):
        exp = SEMANTIC_EXPANSIONS.get(skill, "")
        w = SEMANTIC_WEIGHTS.get(skill, 1)
        boosted.append((" " + exp) * w)

    return " ".join(boosted).strip()

# =========================================
# 5. Motore IR avanzato
# =========================================

def search_players_scout(query: str, top_k: int = 10):
    intents = parse_query_intents(query)
    weighted_query = build_weighted_query(query, intents)

    q_vec = vectorizer.transform([weighted_query])
    sim = cosine_similarity(q_vec, tfidf_matrix).ravel()

    idx = np.argsort(sim)[::-1][:top_k]

    results = metadata_df.iloc[idx].copy()
    results["similarity"] = sim[idx]

    results.attrs["intents"] = intents
    results.attrs["weighted_query"] = weighted_query

    return results

# =========================================
# 6. Formatter numerici
# =========================================

def format_percent_it(value, decimals=1):
    if value is None or pd.isna(value):
        return None
    v = float(value)
    if v <= 1:
        v *= 100
    return f"{v:.{decimals}f}".replace(".", ",") + "%"

def format_number_it(value, decimals=1):
    if value is None or pd.isna(value):
        return None
    return f"{float(value):.{decimals}f}".replace(".", ",")

# =========================================
# 7. Spiegazioni giocatore
# =========================================

def build_explanation(row, intents):
    reasons = []
    name = row.get("Player", "Il giocatore")
    reasons.append(f"{name} è stato selezionato perché:")

    # 3P
    p3 = row.get("3P%")
    if "shooting_3" in intents["skills"] and p3 is not None:
        reasons.append(f"- buon tiratore da tre ({format_percent_it(p3)}).")

    # FT
    ft = row.get("FT%")
    if "ft" in intents["skills"] and ft is not None:
        reasons.append(f"- buon tiratore ai liberi ({format_percent_it(ft)}).")

    # Rimbalzi
    trb_pg = row.get("TRB_per_game") or row.get("TRB.1")
    if "reb" in intents["skills"] and trb_pg is not None:
        reasons.append(f"- buon rimbalzista ({format_number_it(trb_pg)} rimbalzi a partita).")

    # Rimbalzo offensivo
    orb_pg = row.get("ORB_per_game")
    if "reb_off" in intents["skills"] and orb_pg is not None:
        reasons.append(f"- forte rimbalzista offensivo ({format_number_it(orb_pg)} RO).")

    # Assist
    if "playmaking" in intents["skills"] and row.get("AST") is not None:
        reasons.append(f"- buon passatore ({format_number_it(row['AST'])} assist).")

    # Difesa
    if "defense" in intents["skills"]:
        if row.get("DWS") is not None:
            reasons.append("- buon difensore (solidi indici difensivi).")

    if len(reasons) == 1:
        reasons.append("- profilo testuale molto vicino alla descrizione richiesta.")

    return "\n".join(reasons)

# =========================================
# 8. Ranking + spiegazioni
# =========================================

def rank_and_explain(query, top_k=5):
    base = search_players_scout(query, top_k)
    intents = base.attrs["intents"]

    # Se esiste player_id su entrambi, usa quello.
    # Altrimenti fai merge sul nome del giocatore.
    if "player_id" in base.columns and "player_id" in full_df.columns:
        merged = base.merge(full_df, on="player_id", how="left")
    else:
        merged = base.merge(full_df, on="Player", how="left")

    merged["explanation"] = merged.apply(lambda r: build_explanation(r, intents), axis=1)

    return merged, intents


# =========================================
# 9. Stampa formattata
# =========================================

def print_player_result(row, i):
    # Prende nome, anno, pick in modo robusto
    name = row.get("Player", "N/A")

    # DraftYear può essere DraftYear, DraftYear_x o DraftYear_y
    if "DraftYear" in row.index:
        year = row["DraftYear"]
    elif "DraftYear_x" in row.index:
        year = row["DraftYear_x"]
    elif "DraftYear_y" in row.index:
        year = row["DraftYear_y"]
    else:
        year = "N/A"

    # Pick può essere Pick, Pick_x o Pick_y
    if "Pick" in row.index:
        pick = row["Pick"]
    elif "Pick_x" in row.index:
        pick = row["Pick_x"]
    elif "Pick_y" in row.index:
        pick = row["Pick_y"]
    else:
        pick = "N/A"

    header = f"#{i+1} — {name} (Draft {year}, pick {pick})"
    print(header)
    print("-" * len(header))

    explanation = row.get("explanation", "")
    print(explanation)
    print("\n")


# =========================================
# 10. Interfaccia CLI
# =========================================

def run_talent_scout_cli(top_k=5):
    print("=== SISTEMA INTELLIGENTE PER TALENT SCOUT NBA ===")
    print("Scrivi una descrizione del giocatore che cerchi.")
    print("Digita 'exit' per uscire.\n")

    while True:
        query = input("Descrivi il giocatore ideale: ").strip()
        if query.lower() in ("exit", "esci", "quit", "q"):
            print("Chiusura sistema 👋")
            break

        results, intents = rank_and_explain(query, top_k)
        print("\nSkill individuate:", intents["skills"], "\n")

        for i, (_, row) in enumerate(results.iterrows()):
            print_player_result(row, i)
        print("\n")


Indice caricato.
TF-IDF matrix shape: (8323, 11236)
Metadata shape: (8323, 6)
Dataset stats shape: (8323, 35)


In [17]:
run_talent_scout_cli(top_k=5)

=== SISTEMA INTELLIGENTE PER TALENT SCOUT NBA ===
Scrivi una descrizione del giocatore che cerchi.
Digita 'exit' per uscire.


Skill individuate: ['ft', 'reb', 'shooting_3'] 



KeyError: 'DraftYear'

Con questo Step E hai:

trasformato il tuo motore di ranking + spiegazioni in una vera interfaccia da Talent Scout, anche se testuale;

ora un direttore sportivo/scout può:

scrivere in linguaggio naturale il profilo del giocatore ideale,

vedere una shortlist top-N,

leggere una spiegazione in italiano del perché ciascun giocatore è stato proposto.